<a href="https://colab.research.google.com/github/DeependraParichha1004/NLP/blob/main/Text_GEN_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import os
import keras
import numpy as np
import time

In [2]:
path_to_file=tf.keras.utils.get_file('shakespeare.txt','https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1130496/1115394 [==============================] - 0s 0us/step


In [50]:
text=open(path_to_file,'rb').read().decode('utf-8')


In [4]:
len(text)

1115394

In [5]:
text[:250]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n'

In [6]:
vocab=sorted(set(text))
vocab

['\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '3',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [7]:
list(vocab)

['\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '3',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [8]:
char_to_ids=tf.keras.layers.StringLookup(vocabulary=list(vocab),mask_token=None)
char_to_ids

In [9]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [10]:
char_to_ids(chars)

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [11]:
ids =char_to_ids(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [12]:
ids_to_char=tf.keras.layers.StringLookup(vocabulary=char_to_ids.get_vocabulary(),invert=True,mask_token=None)


In [13]:
ids_to_char(ids)

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [14]:
tf.strings.reduce_join(chars,axis=-1)

<tf.Tensor: shape=(2,), dtype=string, numpy=array([b'abcdefg', b'xyz'], dtype=object)>

In [15]:
def ids_to_texts(ids):
  return tf.strings.reduce_join(ids_to_char(ids),axis=-1)

In [17]:
tf.strings.unicode_split(text,'UTF-8')

<tf.Tensor: shape=(1115394,), dtype=string, numpy=array([b'F', b'i', b'r', ..., b'g', b'.', b'\n'], dtype=object)>

In [18]:
type(text)

str

In [19]:
all_ids=char_to_ids(tf.strings.unicode_split(text,'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [20]:
id_dataset=tf.data.Dataset.from_tensor_slices(all_ids)


In [21]:
for i in id_dataset.take(5):
  print(ids_to_char(i).numpy().decode('UTF-8'))

F
i
r
s
t


In [22]:
seq_length=100
sequences=id_dataset.batch(seq_length+1,drop_remainder=True)

In [23]:
sequences

<BatchDataset element_spec=TensorSpec(shape=(101,), dtype=tf.int64, name=None)>

In [24]:
for i in sequences.take(1):
  print(ids_to_char(i).numpy())

[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' ']


In [25]:
def ids_to_text(ids):
  return tf.strings.reduce_join(ids_to_char(ids),axis=-1)

In [26]:
for seq in sequences.take(5):
  print(ids_to_text(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [27]:
def split_input_target(seq):
  inp_text=seq[:-1]
  tar_text=seq[1:]
  return inp_text,tar_text

In [28]:
dataset=sequences.map(split_input_target)

In [29]:
BATCH_SIZE=64
BUFFER_SIZE=10000
dataset=(
    dataset
    .batch(BATCH_SIZE,drop_remainder=True)
    .shuffle(BUFFER_SIZE)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

In [30]:
dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [31]:
def demo(a=3):
  print(a)

demo(4)

4


In [32]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [33]:
vocab_size=len(ids_to_char.get_vocabulary())
embedding_dim=256
rnn_units=1024
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [34]:
for inp_example,target_example in dataset.take(1):
  pred=model(inp_example)
  print(pred[0])

tf.Tensor(
[[-0.00093505 -0.00202025  0.00796145 ... -0.00866009  0.00674443
  -0.00140973]
 [-0.00222641 -0.00764429  0.00439529 ...  0.00443462  0.00023536
   0.00443082]
 [-0.00146869 -0.01644453 -0.00621398 ...  0.0067245   0.01040214
  -0.00222593]
 ...
 [ 0.00229915  0.00894707  0.00927411 ... -0.01441908 -0.00737588
  -0.00700362]
 [-0.00376739  0.00743595 -0.00210765 ... -0.01834096 -0.01290674
  -0.01007859]
 [-0.00461784  0.00117424  0.00483261 ... -0.01777372  0.00195704
  -0.00755087]], shape=(100, 66), dtype=float32)


In [35]:
ids_to_text(inp_example[0])

<tf.Tensor: shape=(), dtype=string, numpy=b' too,\nThat makes himself, but for our honour therein,\nUnworthy thee,--if ever henceforth thou\nThese '>

In [36]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


In [37]:
tf.argmax(pred[0],axis=1)

<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([26, 10, 13, 14, 15, 50, 62, 46, 32, 41, 54, 44, 32, 51, 26, 26, 26,
       26,  2,  2,  2, 26, 59,  7, 46, 46, 46, 46, 37, 26, 46, 14, 52, 26,
       64, 30, 52, 26, 26, 30, 13, 13, 30, 52, 26, 19, 46, 26, 52, 26, 26,
       57, 46, 50, 17, 17, 30, 30, 52, 19, 55, 30, 26, 42, 46, 26, 26, 46,
       52, 52, 63,  2, 26, 26, 62, 26, 62, 26, 26, 26, 26, 26, 26, 26, 14,
       52,  5, 55, 26, 46, 46, 13, 30, 50, 62, 46, 26, 26, 26, 26])>

In [38]:
sampled_indices = tf.random.categorical(pred[0], num_samples=1)
indices=tf.squeeze(sampled_indices,axis=-1).numpy()
indices

array([14,  5, 31, 65, 63, 19, 63, 48, 30, 27,  3, 19, 41, 13, 51, 62, 18,
       48, 42, 60, 14, 64, 22,  2, 60, 20,  8, 22, 61, 22, 42, 30, 28, 63,
        6,  7, 54, 65, 31, 21, 55, 37, 15,  0, 10, 16, 32, 28, 52, 52, 63,
       10, 42, 18, 36, 63, 39, 22, 60, 38, 47,  3, 22,  5, 59,  2, 32, 33,
       36, 64,  6, 56, 19, 53, 49, 24, 26, 39,  4, 33,  2, 57, 16, 46, 41,
       26,  3, 40, 43, 38, 15, 30, 43,  7, 38, 57, 62, 15, 31,  9])

Note: It is important to sample from this distribution as taking the argmax of the distribution can easily get the model stuck in a loop.



In [39]:
print('The input text is:\n',ids_to_text(inp_example[0]).numpy().decode('UTF-8'))

The input text is:
  too,
That makes himself, but for our honour therein,
Unworthy thee,--if ever henceforth thou
These 


In [40]:
print('output is:\n',ids_to_char(indices).numpy())

output is:
 [b'A' b'&' b'R' b'z' b'x' b'F' b'x' b'i' b'Q' b'N' b'!' b'F' b'b' b'?'
 b'l' b'w' b'E' b'i' b'c' b'u' b'A' b'y' b'I' b' ' b'u' b'G' b'-' b'I'
 b'v' b'I' b'c' b'Q' b'O' b'x' b"'" b',' b'o' b'z' b'R' b'H' b'p' b'X'
 b'B' b'[UNK]' b'3' b'C' b'S' b'O' b'm' b'm' b'x' b'3' b'c' b'E' b'W' b'x'
 b'Z' b'I' b'u' b'Y' b'h' b'!' b'I' b'&' b't' b' ' b'S' b'T' b'W' b'y'
 b"'" b'q' b'F' b'n' b'j' b'K' b'M' b'Z' b'$' b'T' b' ' b'r' b'C' b'g'
 b'b' b'M' b'!' b'a' b'd' b'Y' b'B' b'Q' b'd' b',' b'Y' b'r' b'w' b'B'
 b'R' b'.']


In [41]:
batch_loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss=batch_loss(target_example,pred)
loss

<tf.Tensor: shape=(), dtype=float32, numpy=4.1886973>

In [42]:
tf.exp(loss)

<tf.Tensor: shape=(), dtype=float32, numpy=65.936844>

In [43]:
model.compile(optimizer='adam',loss=batch_loss)

In [44]:
#checkpoint
checkpoint_dir='./training_checkpoint'
checkpoint_prefix=os.path.join(checkpoint_dir,'ckpt_{epoch}')
callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,save_weights_only=True)

In [45]:
history=model.fit(dataset,epochs=10,callbacks=[callback])

Epoch 1/10
172/172 [==============================] - 35s 179ms/step - loss: 2.7617
Epoch 2/10
172/172 [==============================] - 33s 179ms/step - loss: 2.0779
Epoch 3/10
172/172 [==============================] - 33s 184ms/step - loss: 1.8249
Epoch 4/10
172/172 [==============================] - 34s 185ms/step - loss: 1.6501
Epoch 5/10
172/172 [==============================] - 34s 185ms/step - loss: 1.5336
Epoch 6/10
172/172 [==============================] - 33s 185ms/step - loss: 1.4545
Epoch 7/10
172/172 [==============================] - 34s 185ms/step - loss: 1.3951
Epoch 8/10
172/172 [==============================] - 33s 180ms/step - loss: 1.3450
Epoch 9/10
172/172 [==============================] - 33s 181ms/step - loss: 1.3025
Epoch 10/10
172/172 [==============================] - 34s 185ms/step - loss: 1.2613


In [47]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [48]:
one_step_model = OneStep(model, ids_to_char,char_to_ids)

In [49]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)


ROMEO:
Which deadly never women art
town, till I incent to thus. If, and fill me with
Franced traitors and tell thou, madam, my Romeo,
Which now to Fally with my us!
Where he known the pantsagons of great power?

LADY ANNE:
Go me, on't, you were none liken.

PETRUCHIO:
That would she at Persolace hash a passop.

BUCKINGHAM:
Ratis fires.

CATESBY:
My lord, I charge thee fair remosses of
great herlort with this virtue: mill'dish!
There are their nert lightnowness: at the lawful is this
thy gracious and morrown myself,
My lord and fretted at ensure; therein myself!
O, how nothing made the warrages: like a too leave!
Preparaintio, get you were told on him;
Do not relame thy witchless; and then at thy dross:
For I'll thrown away tifles a manderous mourthes;
Two, 'tis but which they will somp to see my reiling:
Farewell; by my alliam, I'll cold.

CAPULET:
Brave forbour twenty putticutt, for shemer,s, I can.

Tessen:
When I shall be odd to old this fook ignorant.

NORTHUMBERLAND:
Thou not nex